In [158]:
# video_recommender/chromadb_helper.py

import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from constants import *

# video_recommender/constants.py
# video_recommender/constants.py
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()  # take environment variables from .env.

NEW_VIDEO_WEIGHT = 10
STARTING_WEIGHT = 10
USER_MIN_INTEREST = 4
USER_MAX_INTEREST = 8
COUNT_QUEUED_VIDEOS = 5


# User Actions
LIKE = 1
SHARE = 2
WATCH = 1  # more than 50% of total duration
LOOP = 1

# AI-Related Constants
OPENAI_API_KEY = ""
OPENAI_EmbeddingModel = "text-embedding-3-small"

# ChromaDB
chromadb_name = "Edgur_Video_DB_Vectorstore"

# DB
db_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSAE2tBAnAdXsxk9a9YClFN7MSEVhzEmJD01ewwtooMLxL-Ilod26EbdD8sZeZk0ybiqD-jqT-9RZbn/pub?gid=497214901&single=true&output=csv"  # test spreadsheet
df = pd.read_csv(db_url)

In [159]:
chroma_client = chromadb.PersistentClient(path="db")

def initialize_chromadb():
    """
    Initialize ChromaDB client and collection.

    Returns:
    collection: Initialized ChromaDB collection.
    """

    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
        api_key=OPENAI_API_KEY, model_name=OPENAI_EmbeddingModel
    )
    collection = chroma_client.get_or_create_collection(
        name=chromadb_name,
        embedding_function=openai_ef,
        metadata={"hnsw:space": "cosine"},
    )
    return collection


def add_or_update_chromadb_rows(df, collection):
    """
    Add or update rows in the ChromaDB collection.

    Parameters:
    df (DataFrame): DataFrame containing video data.
    collection: ChromaDB collection.
    """
    documents = df["tags"].apply(lambda x: x.split(",")).tolist()
    documents_str = [", ".join(doc) for doc in documents]
    ids = [str(i + 1) for i in range(len(documents_str))]
    collection.upsert(documents=documents_str, ids=ids)


In [160]:
collection = initialize_chromadb()
add_or_update_chromadb_rows(df, collection)

In [161]:
import openai
import chromadb
from chromadb.config import Settings

# Define template for the system message
system_message_template = """
You are a seasoned developer and computer scientist turned educator, specializing in creating structured learning paths for students of various levels. You have extensive experience in designing curriculums that progress from beginner to intermediate to advanced topics strictly in bullet points. No sub-bullet points. You do not say anything else apart from said bullet points.
"""

# Define template for the user message
user_message_template = "Generate a 10-item-only (no subtopics) curriculum for learning {topic} that progresses from beginner to intermediate to advanced topics."

# Define function to generate curriculum
def generate_curriculum(topic):
    user_message = user_message_template.format(topic=topic)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message_template},
            {"role": "user", "content": user_message}
        ],
        temperature=0.0
    )
    curriculum = response['choices'][0]['message']['content']
    return curriculum.strip().split('\n')

In [163]:
# Initialize a set to store watched video ids
watched_videos = set()

# Retrieve videos for each curriculum item
def retrieve_videos(curriculum, n_results=10):
    global watched_videos
    video_sequence = []

    for item in curriculum:
        results = collection.query(query_texts=item, n_results=n_results)

        for i in range(len(results["ids"][0])):
            result_id = int(results["ids"][0][i])
            if result_id not in watched_videos:
                watched_videos.add(result_id)  # Mark the video as watched

                title = df.loc[df["video_id"] == result_id, "video_title"].values

                video_sequence.append(result_id)
                video_sequence.append(title)
                break  # Break the loop as soon as we find an unwatched video

    return video_sequence

In [164]:
from pprint import pprint

# Main process
topic = "SQL"
curriculum = generate_curriculum(topic)
pprint(curriculum)

['- Introduction to SQL and Relational Databases',
 '- Basic SQL Syntax (SELECT, INSERT, UPDATE, DELETE)',
 '- Filtering Data with WHERE and ORDER BY',
 '- Joining Tables with INNER JOIN',
 '- Aggregating Data with GROUP BY and Aggregate Functions',
 '- Working with Subqueries',
 '- Advanced Joins: OUTER JOIN, SELF JOIN',
 '- Modifying Data with Transactions and Constraints',
 '- Indexing and Performance Optimization',
 '- Advanced SQL Techniques: Window Functions, Common Table Expressions']


In [165]:
video_sequence = retrieve_videos(curriculum)

print("Video Sequence:")
pprint(video_sequence)


Video Sequence:
[525,
 array(['Basic SQL Queries for Data Retrieval'], dtype=object),
 568,
 array(['Implementing Basic Arithmetic Operations in JavaScript'],
      dtype=object),
 38,
 array(['How to Filter Data in Pandas'], dtype=object),
 554,
 array(['Exploring Regular Expressions in Ruby for Data Cleaning'],
      dtype=object),
 77,
 array(['Basic Data Aggregation with Pandas'], dtype=object),
 577,
 array(['Advanced Techniques in Unit Testing in Ruby with unittest'],
      dtype=object),
 544,
 array(['Advanced Techniques in JavaScript ES6 Features'], dtype=object),
 89,
 array(['Understanding Data Normalization'], dtype=object),
 579,
 array(['Advanced Techniques in Ruby List Comprehensions'], dtype=object),
 561,
 array(['Advanced Techniques in JavaScript List Comprehensions'],
      dtype=object)]


### Other Potential Templates

1. **Concept Comparison:**
   - Prompt: "Compare the concepts of {concept1} and {concept2}, highlighting their similarities and differences."
   - Example: "Compare the concepts of SQL and NoSQL, highlighting their similarities and differences."

2. **Brief Overview:**
   - Prompt: "Provide a brief overview of {topic}."
   - Example: "Provide a brief overview of SQL."

3. **Detailed Learning Path (Curriculum):**
   - Prompt: "Generate a 10-item curriculum for learning {topic} that progresses from beginner to intermediate to advanced topics."
   - Example: "Generate a 10-item curriculum for learning Python that progresses from beginner to intermediate to advanced topics."

4. **Step-by-Step Guide:**
   - Prompt: "Provide a step-by-step guide on how to {task}."
   - Example: "Provide a step-by-step guide on how to install and configure MySQL."

5. **Best Practices:**
   - Prompt: "List the best practices for {activity}."
   - Example: "List the best practices for writing SQL queries."

6. **Troubleshooting Tips:**
   - Prompt: "What are the common issues with {topic} and how can they be resolved?"
   - Example: "What are the common issues with database performance and how can they be resolved?"

7. **In-Depth Explanation:**
   - Prompt: "Provide an in-depth explanation of {concept}."
   - Example: "Provide an in-depth explanation of indexing in databases."

8. **Use Case Examples:**
   - Prompt: "Give examples of use cases for {technology or concept}."
   - Example: "Give examples of use cases for machine learning in finance."

In [166]:
def AI_sequence_videos(topic):
    result = collection.query(query_texts=topic, n_results=20)

    # Get the titles of the queried videos
    queried_titles = []
    for i in range(len(result["ids"][0])):
        result_id = int(result["ids"][0][i])
        title = df.loc[df["video_id"] == result_id, "video_title"].values[0]
        queried_titles.append(title)
    pprint(f"Queried Titles: {queried_titles}")  # Debug line to show queried titles
    print("\n")

    user_message = user_message_template.format(topic=topic)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a teacher who lesson-plans effectively. You ensure the sequence of topics you teach is from beginner, then intermediate, then difficult. You respond only with bullet points without sub-bullet points. Work with these content only:" + str(queried_titles)},
            {"role": "user", "content": user_message}
        ],
        temperature=0.0
    )

    return pprint(response)

In [167]:
AI_sequence_videos("SQL")

("Queried Titles: ['Basic SQL Queries for Data Retrieval', 'Understanding Data "
 "Normalization', 'Understanding Data Normalization', 'Implementing Basic "
 "Arithmetic Operations in JavaScript', 'Exploring Regular Expressions in "
 "JavaScript for Data Cleaning', 'Implementing Basic Arithmetic Operations in "
 "Ruby', 'Basic Data Engineering with Apache Spark', 'Basic Data Engineering "
 "with Apache Spark', 'Using the map Function in Ruby for Data "
 "Transformation', 'Using the reduce Function in JavaScript for Data "
 "Aggregation', 'Using the reduce Function in Ruby for Data Aggregation', "
 '\'Using the map Function in JavaScript for Data Transformation\', "Using '
 'JavaScript\'s datetime Module to Handle Dates and Times", "How to Use '
 'Python\'s sqlite3 Library", "How to Use Python\'s sqlite3 Library", "How to '
 'Use Python\'s sqlite3 Library", \'Understanding Chi-Square Test\', '
 "'Understanding Chi-Square Test', 'Basic Principles of Object-Oriented "
 "Programming in Jav